# Linear Decoder Demonstration

This notebook demonstrates how to fit a linear decoder by creating fake data and fitting models to this fake data

In [1]:
import numpy as np
import h5py
import os

In [2]:
# Import linear decoder classes
from linear_decoder_parallelize import LinearDecoderParallelize
from linear_decoder import LinearDecoder

In [3]:
####################
## Make fake data ##
####################  

data_type_vec = ['training','validation','testing']

## Make fake directories 
if not os.path.exists('FakeData'):
    os.makedirs('FakeData')
    for i_data_type in range(len(data_type_vec)):
        os.makedirs('FakeData/'+data_type_vec[i_data_type])
        os.makedirs('FakeData/'+data_type_vec[i_data_type]+'/images')
        os.makedirs('FakeData/'+data_type_vec[i_data_type]+'/activities_spatialSim')
        
if not os.path.exists('FakeModels'):
    os.makedirs('FakeModels')

## Generate fake images and activities

for i_data_type in range(len(data_type_vec)):

    image_dir = 'FakeData/'+data_type_vec[i_data_type]+'/images/'
    activity_dir = 'FakeData/'+data_type_vec[i_data_type]+'/activities_spatialSim/'
    for i_block in range(3): # just create 3 blocks of each data type
        
        # images
        X = np.random.randint(0,255,(10000,256,256))
        save_path = image_dir+str(i_block)+'.h5'
        h5_file = h5py.File(save_path,'w')
        h5_file.create_dataset('data',data=X)
        h5_file.close()
        
        # activities
        Y = np.random.randint(0,255,(10000,41))
        save_path = activity_dir+str(i_block)+'_spatialSim.h5'
        h5_file = h5py.File(save_path,'w')
        h5_file.create_dataset('data',data=Y)
        h5_file.close()


There are two linear decoder classes, LinearDecoderParallelize and LinearDecoder.

LinearDecoder gives you less control and doesn't save any intermediate statistics: it loops through every block in the training data and calculates the linear decoder using least squares. If 'create_decoded_images' is True, it then automatically loops through all the training, validation, and testing data to create and save the linear decoded images.

LinearDecoderParallelize gives you more control over exactly what you're doing and allows you to parallelize on multiple cpus by computing the X^TX and X^TY statistics for different blocks and saving them individually. You can also create the linear decoded images in parallel.

Both of these classes create the same structures: there is a directory created for this linear decoder named using the activity suffix and the string of cell types if fitting to only certain cell types. Within this directory, logs contains text logs of what the linear decoder is doing and the linear decoder weights are contained in model.

In [4]:
### Normal linear decoder
LinearDecoder.runLD(activity_suffix='spatialSim', LD_directory_suffix = 'LD_images_spatialSim', im_width=128, cell_type_list=None, create_decoded_images=True)

In [5]:
### Parallelizable linear decoder
LinearDecoderParallelize.ComputeStatistics(activity_suffix='spatialSim', block_vec = np.asarray([0,2]))

In [6]:
LinearDecoderParallelize.ComputeStatistics(activity_suffix='spatialSim', block_vec = np.asarray([1]))

In [7]:
LinearDecoderParallelize.CreateFullW(activity_suffix='spatialSim', block_vec = np.arange(0,3))

In [8]:
LinearDecoderParallelize.CreateDecodedImages(activity_suffix='spatialSim', LD_directory_suffix = 'LD_images_spatialSim', block_vec = np.arange(0,3),data_type='training')